Import all necessary libraries and install everything you need for training:

In [1]:
# install the libraries necessary for data wrangling, prediction and result analysis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from numba import cuda
from itertools import islice
import time
from tqdm import tqdm

In [2]:
# Install transformers
# (this needs to be done on Kaggle each time you start the session)
#!pip install -q transformers

# Install the simpletransformers
#!pip install -q simpletransformers
from simpletransformers.classification import ClassificationModel

# Install wandb
#!pip install -q wandb
import wandb

# Login to wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: tajak (use `wandb login --relogin` to force relogin)


True

In [3]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [4]:
# Import the MaCoCu sample

dataset = pd.read_csv("MaCoCu-is-sample.csv", sep="\t", index_col = 0)
dataset.head(2)

,domain,url,text,doc,length
0,ft.is,https://ft.is/,Félag Tölvunarfræðinga\nMarkmið félagsins er a...,"<doc id=""macocu.is.97"" title=""Félag Tölvunarfr...",255
1,pfi.is,http://www.pfi.is/,Stefnt er að því að halda þessa fundi í septem...,"<doc id=""macocu.is.142"" title=""Póstmannafélag ...",304


In [5]:
dataset.describe(include="all")

,domain,url,text,doc,length
count,10000,10000,10000,10000,10000.000000
unique,1000,10000,10000,10000,NaN
top,ft.is,https://ft.is/,Félag Tölvunarfræðinga\nMarkmið félagsins er a...,"<doc id=""macocu.is.97"" title=""Félag Tölvunarfr...",NaN
freq,10,1,1,1,NaN
mean,NaN,NaN,NaN,NaN,582.432400
std,NaN,NaN,NaN,NaN,1707.816705
min,NaN,NaN,NaN,NaN,76.000000
25%,NaN,NaN,NaN,NaN,126.000000
50%,NaN,NaN,NaN,NaN,225.000000
75%,NaN,NaN,NaN,NaN,485.000000


## Apply classifier

Models used:
- FTD classifier - original FTD data (except multi-labeled texts and non-text texts) - 10 categories, 849 instances in training data
- GINCO-downcast classifier - used primary_level_4 downcasted GINCO labels - 9 labels. It was trained on 601 texts.
- CORE-main classifier - main categories only - 9 instances. All texts with multiple labels were discarded. It was trained on 10256 instances.
- GINCO X-GENRE classifier - 9 X-GENRE labels. It was trained on 535 texts (10% texts discarded - belonging to "discarded" labels)
- FTD X-GENRE classifier - 7 X-GENRE labels. It was trained on 630 texts (23% texts were discarded).
- CORE X-GENRE classifier - 9 X-GENRE labels. It was trained on 607 texts - large changes to the dataset were performed (change of distribution, taking only a sample to have a similar size as FTD and GINCO).
- X-GENRE classifier - 9 X-GENRE labels. Trained on the training splits of all of the X-GENRE datasets mentioned above: 1772 instances in the training dataset.


### Functions

In [4]:
def define_model(model_name):
	"""
	Define which model you want to use and download it.
	Args:
	- model_name: choose from "FTD", "GINCO", "CORE", "GINCO-X-GENRE", "FTD-X-GENRE", "CORE-X-GENRE", "X-GENRE"
	"""
	if model_name == "FTD":
		# Initialize Wandb: 
		run = wandb.init(project="FTD-learning-manual-hyperparameter-search", entity="tajak", name="applying-predictions-on-MaCoCu")

		# Load the FTD model from Wandb:
		artifact = run.use_artifact('tajak/FTD-learning-manual-hyperparameter-search/FTD-classifier:v1', type='model')
		artifact_dir = artifact.download()

		# Loading a local save
		model = ClassificationModel(
			"xlmroberta", artifact_dir)

	elif model_name == "GINCO":
		# Initialize Wandb: 
		run = wandb.init(project="GINCO-hyperparameter-search", entity="tajak", name="applying-predictions-on-MaCoCu")

		#To load the GINCO-downcast model from Wandb:
		artifact = run.use_artifact('tajak/GINCO-hyperparameter-search/GINCO-downcast-classifier:v0', type='model')
		artifact_dir = artifact.download()

		# Loading a local save
		model = ClassificationModel(
			"xlmroberta", artifact_dir)
	elif model_name == "CORE":
		# Initialize Wandb: 
		run = wandb.init(project="CORE-hyperparameter-search", entity="tajak", name="applying-predictions-on-MaCoCu")

		#To load the CORE-main model from Wandb:
		artifact = run.use_artifact('tajak/CORE-hyperparameter-search/CORE-main-classifier:v0', type='model')
		artifact_dir = artifact.download()

		# Loading a local save
		model = ClassificationModel(
			"xlmroberta", artifact_dir)
	elif model_name == "GINCO-X-GENRE":
		# Initialize Wandb - for X-GENRE classifiers: 
		run = wandb.init(project="X-GENRE classifiers", entity="tajak", name="applying-predictions-on-MaCoCu")

		#To load the GINCO-X-GENRE model from Wandb:
		artifact = run.use_artifact('tajak/X-GENRE classifiers/SI-GINCO-X-GENRE-classifier:v0', type='model')
		artifact_dir = artifact.download()

		# Loading a local save
		model = ClassificationModel(
			"xlmroberta", artifact_dir)
	elif model_name == "FTD-X-GENRE":
		# Initialize Wandb - for X-GENRE classifiers: 
		run = wandb.init(project="X-GENRE classifiers", entity="tajak", name="applying-predictions-on-MaCoCu")

		#To load the FTD-X-GENRE model from Wandb:
		artifact = run.use_artifact('tajak/X-GENRE classifiers/FTD-X-GENRE-classifier:v0', type='model')
		artifact_dir = artifact.download()

		# Loading a local save
		model = ClassificationModel(
			"xlmroberta", artifact_dir)
	elif model_name == "CORE-X-GENRE":
		# Initialize Wandb - for X-GENRE classifiers: 
		run = wandb.init(project="X-GENRE classifiers", entity="tajak", name="applying-predictions-on-MaCoCu")

		# To access the model from the Wandb:
		artifact = run.use_artifact('tajak/X-GENRE classifiers/CORE-X-GENRE-classifier:v0', type='model')
		artifact_dir = artifact.download()

		# Loading a local save
		model = ClassificationModel(
			"xlmroberta", artifact_dir)
	elif model_name == "X-GENRE":
		# Initialize Wandb - for X-GENRE classifiers: 
		run = wandb.init(project="X-GENRE classifiers", entity="tajak", name="applying-predictions-on-MaCoCu")

		# To access the model from the Wandb:
		artifact = run.use_artifact('tajak/X-GENRE classifiers/X-GENRE-classifier:v0', type='model')
		artifact_dir = artifact.download()

		# Loading a local save
		model = ClassificationModel(
			"xlmroberta", artifact_dir)

	return model

In [5]:
def predict(dataset, model, model_name):
	"""
	This function takes the dataset and applies the trained model on it to infer predictions.
	It saves the results to a new csv file.

	Args:
	- dataset (pandas DataFrame): dataset to apply prediction on. The text we want to predict should be in the column "text".
	- model: the model to use
	- model_name: the name of the classifier
	"""
	# Silence the model
	model.args.silent = True

	labels = model.args.labels_list

	# Split the dataframe into batches

	def chunk(arr_range, arr_size):
		arr_range = iter(arr_range)
		return iter(lambda: tuple(islice(arr_range, arr_size)), ())

	batches_list = list(chunk(dataset.text, 8))

	batches_list_new = []

	for i in batches_list:
		batches_list_new.append(list(i))

	print("The dataset is split into {} batches of {} texts.".format(len(batches_list_new),len(batches_list_new[0])))

	# Apply softmax to the raw outputs
	def softmax(x):
		#Compute softmax values for each sets of scores in x.
		return np.exp(x) / np.sum(np.exp(x), axis=0)

	y_pred = []
	y_distr = []
	most_probable = []
	batch_counter = 0

	print(f"Prediction with model {model_name} started.")
	start_time = time.time()

	for i in tqdm(batches_list_new):
		output = model.predict(i)
		current_y_pred = output[0]
		current_y_distr = output[1]
		current_y_distr_softmax = []
		current_y_distr_most_probable = []
		for i in current_y_distr:
			distr = softmax(i)
			distr_dict = {labels[i]: round(distr[i],4) for i in range(len(labels))}
			current_y_distr_softmax.append(distr_dict)
			# Also add the information for the softmax of the most probable category ("certainty")
			distr_sorted = np.sort(distr)
			current_y_distr_most_probable.append(distr_sorted[-1])

		for i in current_y_pred:
			y_pred.append(i)

		for i in current_y_distr_softmax:
			y_distr.append(i)

		for i in current_y_distr_most_probable:
			most_probable.append(i)

		batch_counter += 1
		#print("Batch {} predicted.".format(batch_counter))

	prediction_time = round((time.time() - start_time)/60,2)

	print("Prediction with model {} completed. It took {} minutes for {} instances - {} minutes per one instance.".format(model_name, prediction_time, dataset.shape[0], prediction_time/dataset.shape[0]))

	dataset[f"{model_name}"] = y_pred
	dataset[f"{model_name}_label_distribution"] = y_distr
	dataset[f"{model_name}_confidence"] = most_probable

	# Save the new dataframe which contains the y_pred values as well
	dataset.to_csv("MaCoCu-sl-sample-prediction-{}".format(model_name), sep="\t")

	return dataset

### Prediction

#### FTD

In [8]:
FTD = define_model("FTD")

# Define labels
FTD_labels = [7, 8, 0, 1, 6, 5, 2, 4, 3, 9]

ftd_mapping = {'A1 (argumentative)': 0, 'A11 (personal)': 1, 'A12 (promotion)': 2, 'A14 (academic)': 3, 'A16 (information)': 4, 'A17 (review)': 5, 'A4 (fiction)': 6, 'A7 (instruction)': 7, 'A8 (news)': 8, 'A9 (legal)': 9}

ftd_mapping_reverse = {list(ftd_mapping.values())[i]: list(ftd_mapping.keys())[i] for i in range(len(list(ftd_mapping.values())))}

wandb: Downloading large artifact FTD-classifier:v1, 1081.90MB. 8 files... Done. 0:0:0
/home/tajak/anaconda3/lib/python3.9/site-packages/simpletransformers/classification/classification_model.py:459: UserWarning: use_multiprocessing automatically disabled as xlmroberta fails when using multiprocessing for feature conversion.
  warnings.warn(


In [9]:
# Predict FTD to the dataset
dataset = predict(dataset, FTD, "FTD")

The dataset is split into 1250 batches of 8 texts.
Prediction with model FTD started.


100%|██████████| 1250/1250 [23:05<00:00,  1.11s/it]


Prediction with model FTD completed. It took 23.09 minutes for 10000 instances - 0.002309 minutes per one instance.


In [10]:
dataset.head(3)

,domain,url,text,doc,length,FTD,FTD_label_distribution,FTD_confidence
0,ft.is,https://ft.is/,Félag Tölvunarfræðinga\nMarkmið félagsins er a...,"<doc id=""macocu.is.97"" title=""Félag Tölvunarfr...",255,2,"{7: 0.0106, 8: 0.0275, 0: 0.0294, 1: 0.0066, 6...",0.853838
1,pfi.is,http://www.pfi.is/,Stefnt er að því að halda þessa fundi í septem...,"<doc id=""macocu.is.142"" title=""Póstmannafélag ...",304,2,"{7: 0.029, 8: 0.0436, 0: 0.0413, 1: 0.0094, 6:...",0.679917
2,sfa.is,https://sfa.is/,Samtök forstöðumanna almenningsbókasafna\nSamt...,"<doc id=""macocu.is.143"" title=""Samtök forstöðu...",421,4,"{7: 0.0206, 8: 0.0319, 0: 0.0201, 1: 0.0108, 6...",0.778097


In [11]:
# Map the FTD labels to names
dataset["FTD"] = [ftd_mapping_reverse[x] for x in dataset["FTD"]]

dataset.head(3)

,domain,url,text,doc,length,FTD,FTD_label_distribution,FTD_confidence
0,ft.is,https://ft.is/,Félag Tölvunarfræðinga\nMarkmið félagsins er a...,"<doc id=""macocu.is.97"" title=""Félag Tölvunarfr...",255,A12 (promotion),"{7: 0.0106, 8: 0.0275, 0: 0.0294, 1: 0.0066, 6...",0.853838
1,pfi.is,http://www.pfi.is/,Stefnt er að því að halda þessa fundi í septem...,"<doc id=""macocu.is.142"" title=""Póstmannafélag ...",304,A12 (promotion),"{7: 0.029, 8: 0.0436, 0: 0.0413, 1: 0.0094, 6:...",0.679917
2,sfa.is,https://sfa.is/,Samtök forstöðumanna almenningsbókasafna\nSamt...,"<doc id=""macocu.is.143"" title=""Samtök forstöðu...",421,A16 (information),"{7: 0.0206, 8: 0.0319, 0: 0.0201, 1: 0.0108, 6...",0.778097


In [12]:
# At the end of each prediction and before downloading the next model, delete the previous one from the folder to release space.

# Remove previous classifier
%rm -rf wandb
%rm -rf artifacts

In [13]:
dataset["FTD"].value_counts()

A12 (promotion)       3054
A16 (information)     2028
A1 (argumentative)    1916
A8 (news)              910
A11 (personal)         806
A17 (review)           516
A7 (instruction)       438
A9 (legal)             241
A4 (fiction)            62
A14 (academic)          29
Name: FTD, dtype: int64

In [14]:
# Save the dataset with results
dataset.to_csv("MaCoCu-is_with_predictions.csv", sep="\t")

#### Loop through other classifiers

In [6]:
# Open the dataset
dataset = pd.read_csv("MaCoCu-sl-sample-prediction-GINCO-X-GENRE", sep="\t", index_col = 0)
dataset.head(2)

,domain,url,text,doc,length,FTD,FTD_label_distribution,FTD_confidence,GINCO,GINCO_label_distribution,GINCO_confidence,CORE,CORE_label_distribution,CORE_confidence,GINCO-X-GENRE,GINCO-X-GENRE_label_distribution,GINCO-X-GENRE_confidence
0,ft.is,https://ft.is/,Félag Tölvunarfræðinga\nMarkmið félagsins er a...,"<doc id=""macocu.is.97"" title=""Félag Tölvunarfr...",255,A12 (promotion),"{7: 0.0106, 8: 0.0275, 0: 0.0294, 1: 0.0066, 6...",0.853838,List of Summaries/Excerpts,"{'Forum': 0.0057, 'News/Reporting': 0.0322, 'O...",0.894127,Informational Description/Explanation,"{'Interactive Discussion': 0.0019, 'Narrative'...",0.972140,News,"{'Information/Explanation': 0.0036, 'Opinion/A...",0.933308
1,pfi.is,http://www.pfi.is/,Stefnt er að því að halda þessa fundi í septem...,"<doc id=""macocu.is.142"" title=""Póstmannafélag ...",304,A12 (promotion),"{7: 0.029, 8: 0.0436, 0: 0.0413, 1: 0.0094, 6:...",0.679917,Information/Explanation,"{'Forum': 0.0363, 'News/Reporting': 0.0227, 'O...",0.522654,Informational Description/Explanation,"{'Interactive Discussion': 0.0021, 'Narrative'...",0.968317,Information/Explanation,"{'Information/Explanation': 0.9309, 'Opinion/A...",0.930899


In [7]:
def prediction_pipeline(model_name, dataset):
	"""Choose from ["GINCO", "CORE", "GINCO-X-GENRE", "FTD-X-GENRE", "CORE-X-GENRE", "X-GENRE"]"""

	# Define the model
	model = define_model(model_name)

	# Predict genre labels to the dataset
	dataset = predict(dataset, model, model_name)

	# Before downloading the next model, delete the previous one to release space.
	%rm -rf wandb
	%rm -rf artifacts

In [15]:
prediction_pipeline("GINCO", dataset)

Thread SenderThread:
Traceback (most recent call last):
  File "/home/tajak/anaconda3/lib/python3.9/site-packages/wandb/sdk/internal/internal_util.py", line 54, in run
    self._run()
  File "/home/tajak/anaconda3/lib/python3.9/site-packages/wandb/sdk/internal/internal_util.py", line 105, in _run
    self._process(record)
  File "/home/tajak/anaconda3/lib/python3.9/site-packages/wandb/sdk/internal/internal.py", line 312, in _process
    self._sm.send(record)
  File "/home/tajak/anaconda3/lib/python3.9/site-packages/wandb/sdk/internal/sender.py", line 237, in send
    send_handler(record)
  File "/home/tajak/anaconda3/lib/python3.9/site-packages/wandb/sdk/internal/sender.py", line 375, in send_exit
    self._update_summary()
  File "/home/tajak/anaconda3/lib/python3.9/site-packages/wandb/sdk/internal/sender.py", line 842, in _update_summary
    with open(summary_path, "w") as f:
FileNotFoundError: [Errno 2] No such file or directory: '/home/tajak/Genre-Datasets-Comparison/Genre-Datasets

In [18]:
dataset.head(2)

,domain,url,text,doc,length,FTD,FTD_label_distribution,FTD_confidence,GINCO,GINCO_label_distribution,GINCO_confidence
0,gg.mk,https://gg.mk/,"Тоа е тоа, екипа. Претпоследната епизода ја на...","<doc id=""macocu.mk.4"" title=""GG.MK - е-спорт &...",864,A17 (review),"{7: 0.0152, 8: 0.1352, 0: 0.0334, 1: 0.0402, 6...",0.643622,List of Summaries/Excerpts,"{'Forum': 0.0036, 'News/Reporting': 0.0147, 'O...",0.919893
1,qs.mk,https://www.qs.mk/,"Парадата на гордоста се враќа под слоганот ""Во...","<doc id=""macocu.mk.5"" title=""КВИР СКВЕР Скопје...",1054,A12 (promotion),"{7: 0.0056, 8: 0.0162, 0: 0.0263, 1: 0.0047, 6...",0.904037,List of Summaries/Excerpts,"{'Forum': 0.0037, 'News/Reporting': 0.0111, 'O...",0.780482


In [19]:
dataset["GINCO"].value_counts()

News/Reporting                3136
Promotion                     2439
Information/Explanation       1772
List of Summaries/Excerpts     835
Opinion/Argumentation          827
Instruction                    488
Other                          322
Forum                          108
Legal/Regulation                73
Name: GINCO, dtype: int64

In [21]:
prediction_pipeline("CORE", dataset)

wandb: Downloading large artifact CORE-main-classifier:v0, 1081.88MB. 8 files... Done. 0:0:0
/home/tajak/anaconda3/lib/python3.9/site-packages/simpletransformers/classification/classification_model.py:459: UserWarning: use_multiprocessing automatically disabled as xlmroberta fails when using multiprocessing for feature conversion.
  warnings.warn(


The dataset is split into 1250 batches of 8 texts.
Prediction with model CORE started.


100%|██████████| 1250/1250 [22:33<00:00,  1.08s/it]


Prediction with model CORE completed. It took 22.56 minutes for 10000 instances - 0.0022559999999999998 minutes per one instance.


In [22]:
dataset.head(2)

,domain,url,text,doc,length,FTD,FTD_label_distribution,FTD_confidence,GINCO,GINCO_label_distribution,GINCO_confidence,CORE,CORE_label_distribution,CORE_confidence
0,gg.mk,https://gg.mk/,"Тоа е тоа, екипа. Претпоследната епизода ја на...","<doc id=""macocu.mk.4"" title=""GG.MK - е-спорт &...",864,A17 (review),"{7: 0.0152, 8: 0.1352, 0: 0.0334, 1: 0.0402, 6...",0.643622,List of Summaries/Excerpts,"{'Forum': 0.0036, 'News/Reporting': 0.0147, 'O...",0.919893,Opinion,"{'Interactive Discussion': 0.0085, 'Narrative'...",0.541413
1,qs.mk,https://www.qs.mk/,"Парадата на гордоста се враќа под слоганот ""Во...","<doc id=""macocu.mk.5"" title=""КВИР СКВЕР Скопје...",1054,A12 (promotion),"{7: 0.0056, 8: 0.0162, 0: 0.0263, 1: 0.0047, 6...",0.904037,List of Summaries/Excerpts,"{'Forum': 0.0037, 'News/Reporting': 0.0111, 'O...",0.780482,Informational Description/Explanation,"{'Interactive Discussion': 0.001, 'Narrative':...",0.950632


In [23]:
dataset["CORE"].value_counts()

Informational Description/Explanation    5809
Narrative                                2376
Informational Persuasion                  736
Opinion                                   448
How-To/Instructional                      417
Interactive Discussion                     92
Spoken                                     82
Lyrical                                    40
Name: CORE, dtype: int64

In [25]:
prediction_pipeline("GINCO-X-GENRE", dataset)

wandb: Downloading large artifact SI-GINCO-X-GENRE-classifier:v0, 1081.88MB. 44 files... Done. 0:0:0
/home/tajak/anaconda3/lib/python3.9/site-packages/simpletransformers/classification/classification_model.py:459: UserWarning: use_multiprocessing automatically disabled as xlmroberta fails when using multiprocessing for feature conversion.
  warnings.warn(


The dataset is split into 1250 batches of 8 texts.
Prediction with model GINCO-X-GENRE started.


100%|██████████| 1250/1250 [23:19<00:00,  1.12s/it]


Prediction with model GINCO-X-GENRE completed. It took 23.32 minutes for 10000 instances - 0.002332 minutes per one instance.


In [26]:
dataset.head(2)

,domain,url,text,doc,length,FTD,FTD_label_distribution,FTD_confidence,GINCO,GINCO_label_distribution,GINCO_confidence,CORE,CORE_label_distribution,CORE_confidence,GINCO-X-GENRE,GINCO-X-GENRE_label_distribution,GINCO-X-GENRE_confidence
0,gg.mk,https://gg.mk/,"Тоа е тоа, екипа. Претпоследната епизода ја на...","<doc id=""macocu.mk.4"" title=""GG.MK - е-спорт &...",864,A17 (review),"{7: 0.0152, 8: 0.1352, 0: 0.0334, 1: 0.0402, 6...",0.643622,List of Summaries/Excerpts,"{'Forum': 0.0036, 'News/Reporting': 0.0147, 'O...",0.919893,Opinion,"{'Interactive Discussion': 0.0085, 'Narrative'...",0.541413,Opinion/Argumentation,"{'Information/Explanation': 0.0055, 'Opinion/A...",0.861316
1,qs.mk,https://www.qs.mk/,"Парадата на гордоста се враќа под слоганот ""Во...","<doc id=""macocu.mk.5"" title=""КВИР СКВЕР Скопје...",1054,A12 (promotion),"{7: 0.0056, 8: 0.0162, 0: 0.0263, 1: 0.0047, 6...",0.904037,List of Summaries/Excerpts,"{'Forum': 0.0037, 'News/Reporting': 0.0111, 'O...",0.780482,Informational Description/Explanation,"{'Interactive Discussion': 0.001, 'Narrative':...",0.950632,Promotion,"{'Information/Explanation': 0.0014, 'Opinion/A...",0.990890


In [8]:
dataset["GINCO-X-GENRE"].value_counts(normalize=True)

Information/Explanation    0.2489
Promotion                  0.2174
Opinion/Argumentation      0.1408
News                       0.1384
Other                      0.1245
Instruction                0.0634
Forum                      0.0478
Legal                      0.0145
Prose/Lyrical              0.0043
Name: GINCO-X-GENRE, dtype: float64

In [9]:
prediction_pipeline("FTD-X-GENRE", dataset)

wandb: Downloading large artifact FTD-X-GENRE-classifier:v0, 1081.88MB. 44 files... Done. 0:0:0
/home/tajak/anaconda3/lib/python3.9/site-packages/simpletransformers/classification/classification_model.py:459: UserWarning: use_multiprocessing automatically disabled as xlmroberta fails when using multiprocessing for feature conversion.
  warnings.warn(


The dataset is split into 1250 batches of 8 texts.
Prediction with model FTD-X-GENRE started.


100%|██████████| 1250/1250 [21:04<00:00,  1.01s/it]


Prediction with model FTD-X-GENRE completed. It took 21.07 minutes for 10000 instances - 0.002107 minutes per one instance.


In [10]:
dataset.head(2)

,domain,url,text,doc,length,FTD,FTD_label_distribution,FTD_confidence,GINCO,GINCO_label_distribution,GINCO_confidence,CORE,CORE_label_distribution,CORE_confidence,GINCO-X-GENRE,GINCO-X-GENRE_label_distribution,GINCO-X-GENRE_confidence,FTD-X-GENRE,FTD-X-GENRE_label_distribution,FTD-X-GENRE_confidence
0,ft.is,https://ft.is/,Félag Tölvunarfræðinga\nMarkmið félagsins er a...,"<doc id=""macocu.is.97"" title=""Félag Tölvunarfr...",255,A12 (promotion),"{7: 0.0106, 8: 0.0275, 0: 0.0294, 1: 0.0066, 6...",0.853838,List of Summaries/Excerpts,"{'Forum': 0.0057, 'News/Reporting': 0.0322, 'O...",0.894127,Informational Description/Explanation,"{'Interactive Discussion': 0.0019, 'Narrative'...",0.972140,News,"{'Information/Explanation': 0.0036, 'Opinion/A...",0.933308,Promotion,"{'Promotion': 0.6432, 'Opinion/Argumentation':...",0.643164
1,pfi.is,http://www.pfi.is/,Stefnt er að því að halda þessa fundi í septem...,"<doc id=""macocu.is.142"" title=""Póstmannafélag ...",304,A12 (promotion),"{7: 0.029, 8: 0.0436, 0: 0.0413, 1: 0.0094, 6:...",0.679917,Information/Explanation,"{'Forum': 0.0363, 'News/Reporting': 0.0227, 'O...",0.522654,Informational Description/Explanation,"{'Interactive Discussion': 0.0021, 'Narrative'...",0.968317,Information/Explanation,"{'Information/Explanation': 0.9309, 'Opinion/A...",0.930899,Promotion,"{'Promotion': 0.8225, 'Opinion/Argumentation':...",0.822453


In [11]:
dataset["FTD-X-GENRE"].value_counts(normalize=True)

Promotion                  0.3636
Information/Explanation    0.2741
News                       0.1654
Opinion/Argumentation      0.1041
Instruction                0.0396
Legal                      0.0351
Prose/Lyrical              0.0181
Name: FTD-X-GENRE, dtype: float64

In [13]:
prediction_pipeline("CORE-X-GENRE", dataset)

wandb: Downloading large artifact CORE-X-GENRE-classifier:v0, 1081.88MB. 8 files... Done. 0:0:0
/home/tajak/anaconda3/lib/python3.9/site-packages/simpletransformers/classification/classification_model.py:459: UserWarning: use_multiprocessing automatically disabled as xlmroberta fails when using multiprocessing for feature conversion.
  warnings.warn(


The dataset is split into 1250 batches of 8 texts.
Prediction with model CORE-X-GENRE started.


100%|██████████| 1250/1250 [23:00<00:00,  1.10s/it]


Prediction with model CORE-X-GENRE completed. It took 23.01 minutes for 10000 instances - 0.002301 minutes per one instance.


In [14]:
dataset.head(2)

,domain,url,text,doc,length,FTD,FTD_label_distribution,FTD_confidence,GINCO,GINCO_label_distribution,...,CORE_confidence,GINCO-X-GENRE,GINCO-X-GENRE_label_distribution,GINCO-X-GENRE_confidence,FTD-X-GENRE,FTD-X-GENRE_label_distribution,FTD-X-GENRE_confidence,CORE-X-GENRE,CORE-X-GENRE_label_distribution,CORE-X-GENRE_confidence
0,ft.is,https://ft.is/,Félag Tölvunarfræðinga\nMarkmið félagsins er a...,"<doc id=""macocu.is.97"" title=""Félag Tölvunarfr...",255,A12 (promotion),"{7: 0.0106, 8: 0.0275, 0: 0.0294, 1: 0.0066, 6...",0.853838,List of Summaries/Excerpts,"{'Forum': 0.0057, 'News/Reporting': 0.0322, 'O...",...,0.972140,News,"{'Information/Explanation': 0.0036, 'Opinion/A...",0.933308,Promotion,"{'Promotion': 0.6432, 'Opinion/Argumentation':...",0.643164,Information/Explanation,"{'Other': 0.048, 'Information/Explanation': 0....",0.516619
1,pfi.is,http://www.pfi.is/,Stefnt er að því að halda þessa fundi í septem...,"<doc id=""macocu.is.142"" title=""Póstmannafélag ...",304,A12 (promotion),"{7: 0.029, 8: 0.0436, 0: 0.0413, 1: 0.0094, 6:...",0.679917,Information/Explanation,"{'Forum': 0.0363, 'News/Reporting': 0.0227, 'O...",...,0.968317,Information/Explanation,"{'Information/Explanation': 0.9309, 'Opinion/A...",0.930899,Promotion,"{'Promotion': 0.8225, 'Opinion/Argumentation':...",0.822453,Information/Explanation,"{'Other': 0.0364, 'Information/Explanation': 0...",0.446269


In [15]:
dataset["CORE-X-GENRE"].value_counts(normalize=True)

Information/Explanation    0.3745
News                       0.1791
Opinion/Argumentation      0.1619
Forum                      0.1398
Instruction                0.0873
Prose/Lyrical              0.0371
Other                      0.0203
Name: CORE-X-GENRE, dtype: float64

In [17]:
prediction_pipeline("X-GENRE", dataset)

wandb: Downloading large artifact X-GENRE-classifier:v0, 1081.88MB. 8 files... Done. 0:0:0
/home/tajak/anaconda3/lib/python3.9/site-packages/simpletransformers/classification/classification_model.py:459: UserWarning: use_multiprocessing automatically disabled as xlmroberta fails when using multiprocessing for feature conversion.
  warnings.warn(


The dataset is split into 1250 batches of 8 texts.
Prediction with model X-GENRE started.


100%|██████████| 1250/1250 [23:00<00:00,  1.10s/it]


Prediction with model X-GENRE completed. It took 23.0 minutes for 10000 instances - 0.0023 minutes per one instance.


In [18]:
# View the final dataset
dataset.describe(include="all")

,domain,url,text,doc,length,FTD,FTD_label_distribution,FTD_confidence,GINCO,GINCO_label_distribution,...,GINCO-X-GENRE_confidence,FTD-X-GENRE,FTD-X-GENRE_label_distribution,FTD-X-GENRE_confidence,CORE-X-GENRE,CORE-X-GENRE_label_distribution,CORE-X-GENRE_confidence,X-GENRE,X-GENRE_label_distribution,X-GENRE_confidence
count,10000,10000,10000,10000,10000.000000,10000,10000,10000.000000,10000,10000,...,10000.000000,10000,10000,10000.000000,10000,10000,10000.000000,10000,10000,10000.000000
unique,1000,10000,10000,10000,NaN,10,10000,NaN,9,10000,...,NaN,7,10000,NaN,7,10000,NaN,9,5929,NaN
top,ft.is,https://ft.is/,Félag Tölvunarfræðinga\nMarkmið félagsins er a...,"<doc id=""macocu.is.97"" title=""Félag Tölvunarfr...",NaN,A12 (promotion),"{7: 0.0106, 8: 0.0275, 0: 0.0294, 1: 0.0066, 6...",NaN,Information/Explanation,"{'Forum': 0.0057, 'News/Reporting': 0.0322, 'O...",...,NaN,Promotion,"{'Promotion': 0.6432, 'Opinion/Argumentation':...",NaN,Information/Explanation,"{'Other': 0.048, 'Information/Explanation': 0....",NaN,Information/Explanation,"{'Other': 0.0001, 'Information/Explanation': 0...",NaN
freq,10,1,1,1,NaN,3054,1,NaN,2468,1,...,NaN,3636,1,NaN,3745,1,NaN,2159,502,NaN
mean,NaN,NaN,NaN,NaN,582.432400,NaN,NaN,0.666613,NaN,NaN,...,0.889176,NaN,NaN,0.653579,NaN,NaN,0.597616,NaN,NaN,0.965467
std,NaN,NaN,NaN,NaN,1707.816705,NaN,NaN,0.228173,NaN,NaN,...,0.155213,NaN,NaN,0.243003,NaN,NaN,0.228331,NaN,NaN,0.094860
min,NaN,NaN,NaN,NaN,76.000000,NaN,NaN,0.147316,NaN,NaN,...,0.281934,NaN,NaN,0.167343,NaN,NaN,0.157022,NaN,NaN,0.316487
25%,NaN,NaN,NaN,NaN,126.000000,NaN,NaN,0.461602,NaN,NaN,...,0.863001,NaN,NaN,0.433896,NaN,NaN,0.391631,NaN,NaN,0.993155
50%,NaN,NaN,NaN,NaN,225.000000,NaN,NaN,0.697739,NaN,NaN,...,0.965129,NaN,NaN,0.673161,NaN,NaN,0.591244,NaN,NaN,0.998366
75%,NaN,NaN,NaN,NaN,485.000000,NaN,NaN,0.893767,NaN,NaN,...,0.988434,NaN,NaN,0.899568,NaN,NaN,0.822026,NaN,NaN,0.998928


In [19]:
dataset["X-GENRE"].value_counts()

Information/Explanation    2159
News                       2049
Opinion/Argumentation      2016
Promotion                  1634
Forum                       677
Instruction                 611
Other                       501
Legal                       247
Prose/Lyrical               106
Name: X-GENRE, dtype: int64

In [20]:
dataset.columns

Index(['domain', 'url', 'text', 'doc', 'length', 'FTD',
       'FTD_label_distribution', 'FTD_confidence', 'GINCO',
       'GINCO_label_distribution', 'GINCO_confidence', 'CORE',
       'CORE_label_distribution', 'CORE_confidence', 'GINCO-X-GENRE',
       'GINCO-X-GENRE_label_distribution', 'GINCO-X-GENRE_confidence',
       'FTD-X-GENRE', 'FTD-X-GENRE_label_distribution',
       'FTD-X-GENRE_confidence', 'CORE-X-GENRE',
       'CORE-X-GENRE_label_distribution', 'CORE-X-GENRE_confidence', 'X-GENRE',
       'X-GENRE_label_distribution', 'X-GENRE_confidence'],
      dtype='object')

In [21]:
# Save the final dataset with results
dataset.to_csv("MaCoCu-is_with_predictions.csv", sep="\t")